In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import asarray
from numpy import savetxt
import statsmodels.api as sm
from scipy import stats
from scipy.optimize import curve_fit
import scipy
from platform import python_version

In [ ]:
print(python_version())

In [ ]:
def inches_to_cm(inches):
    return inches/2.54

In [ ]:
def asterisks(v, ns):
    if((v<0.001)):
        significance='***'
    elif((v<0.01)):
        significance='**'
    elif((v<0.05)):
        significance='*'
    elif((v>=0.05)):
        if(ns):
            significance='ns'
        else:
            significance=''
    return significance

In [ ]:
def LR(coefficients_names, coefficients_present, df, gene, ylim, name, dimensions, df_change):
    
    output_directory = r"output\\"
    
    df1 = df.copy()
    df2 = df_change.copy()

    max_lim = ylim[0]
    min_lim = ylim[1]

    fig, (ax1) = plt.subplots(1, 1, figsize=(inches_to_cm(dimensions[0]),inches_to_cm(dimensions[1])))
    
    error_kw=dict(lw=1.25* dimensions[1]/14.5, capsize=4* dimensions[1]/14.5, capthick=1.25* dimensions[1]/14.5)
    
##############################################################################################################    
        
    LR_model = sm.OLS(df1[gene], sm.add_constant(df1[coefficients_names].astype(int)), missing='drop')
    results = LR_model.fit()

    LR_change = sm.OLS(df2[gene], sm.add_constant(df2[coefficients_names].astype(int)), missing='drop')
    results_change = LR_change.fit()
    print(results_change.summary())
    
    coefficients = results.params[coefficients_present]
    coef_err = results.bse[coefficients_present]
    
    ax1.set_ylim([min_lim, max_lim])
    
    ax1.bar(x=coefficients_present, height=coefficients, color='gray', yerr=coef_err, ecolor='black', error_kw=error_kw)
    ax1.axhline(y=0, color='k', linestyle='--', linewidth=2* dimensions[1]/14.5, label="unvaccinated")
    
    ax1.tick_params(axis='x', direction='in', length=3, labelsize=9* dimensions[1]/14.5, rotation=75)
    ax1.tick_params(axis='y', direction='in', length=3, labelsize=9* dimensions[1]/14.5, right=True)

    ax1.set_ylabel('Ct regression coefficients, ' + gene, fontsize=10* dimensions[1]/14.5)
    ax1.set_xlabel("Vaccination status", fontsize=10* dimensions[1]/14.5, loc='center')
    significance = ''
    
    flag_sig = 0
    for i, v in enumerate(results.pvalues[coefficients_present]):
        if(i<len(coefficients)-1):
            print('p-value change ', results_change.pvalues[coefficients_present][i+1])
            if(results_change.pvalues[coefficients_present][i+1]<0.05):
                significance_change = asterisks(results_change.pvalues[coefficients_present][i+1], False)
                flag_sig += 0.3
                
                if(coefficients[i] + coef_err[i] >= coefficients[i+1] + coef_err[i+1]):
                    height_sig = coefficients[i] + coef_err[i] + 0.01*(max_lim-min_lim) + 0.4 + flag_sig
                    plt.hlines(y=height_sig, xmin=(i+0), xmax=(i+1), color='k', linestyle='-', linewidth=2* dimensions[1]/14.5)
                    plt.vlines(x=i, ymin=height_sig-0.2, ymax=height_sig, color='k', linestyle='-', linewidth=2* dimensions[1]/14.5)
                    plt.vlines(x=i+1, ymin=height_sig-0.6, ymax=height_sig, color='k', linestyle='-', linewidth=2* dimensions[1]/14.5)

                elif(coefficients[i] + coef_err[i] < coefficients[i+1] + coef_err[i+1]):
                    height_sig = coefficients[i+1] + coef_err[i+1] + 0.01*(max_lim-min_lim) + 0.4 + flag_sig
                    plt.hlines(y=height_sig, xmin=(i+0), xmax=(i+1), color='k', linestyle='-', linewidth=2* dimensions[1]/14.5)
                    plt.vlines(x=i, ymin=height_sig-0.6, ymax=height_sig, color='k', linestyle='-', linewidth=2* dimensions[1]/14.5)
                    plt.vlines(x=i+1, ymin=height_sig-0.2, ymax=height_sig, color='k', linestyle='-', linewidth=2* dimensions[1]/14.5)
                
                plt.text(x=i+0.5,
                y=height_sig + 0.2,
                s=significance_change,
                color='black',
                horizontalalignment='center', fontsize=10* dimensions[1]/14.5)
            else:
                flag_sig = 0

        if(coefficients[i] > 0):
            height = coefficients[i] + coef_err[i] + 0.01*(max_lim-min_lim)
        else:
            height = coefficients[i] - coef_err[i] - 0.04*(max_lim-min_lim)
        significance = asterisks(v, True)   
        ax1.text(x=i,
            y=height,
            s=significance,
            color='black',
            horizontalalignment='center', fontsize=10* dimensions[1]/14.5)
        
        export_array_panel_1 = pd.DataFrame(np.array([np.array(range(1,len(coefficients)+1)),np.array(coefficients), np.array(coef_err)]).T, columns=['column #', 'coefficients', 'coefficients error'])
        export_array_panel_1.to_csv(output_directory + r"csv\\" + name + ' - ' + gene + '.csv')
        
        
    ax1.legend(fontsize=10* dimensions[1]/14.5, frameon=False)
    fig.savefig(output_directory + r"png\\" + name + ' - ' + gene + '.png', dpi=600, bbox_inches='tight')
    fig.savefig(output_directory + r"eps\\" + name + ' - ' + gene + '.eps', dpi=600, bbox_inches='tight')
    fig.savefig(output_directory + r"jpg\\" + name + ' - ' + gene + '.jpg', dpi=600, bbox_inches='tight')
    fig.savefig(output_directory + r"tif\\" + name + ' - ' + gene + '.tif', dpi=600, bbox_inches='tight')
    fig.savefig(output_directory + r"svg\\" + name + ' - ' + gene + '.svg', dpi=600, bbox_inches='tight')

In [ ]:
def binning_names(bins, booster):
    names = []
    if(booster):
        for i in range(len(bins)-1):
            names.append(str(bins[i]+1) + '-' + str(bins[i+1]) + ' after booster')
    else:
        for i in range(len(bins)-1):
            names.append(str(bins[i]+1) + '-' + str(bins[i+1]) + ' after 2nd shot')
    if(booster==False):
        names.append('>' + str(bins[len(bins)-1]) + ' after 2nd shot')
    return names

In [ ]:
def binning(df, bins, zeros, comparing, booster):
    bin_names = binning_names(bins, booster)
    if(zeros==False):
        for i in range(len(bins)-1):
            if(comparing==False):
                if(booster):
                    df[bin_names[i]] = df['time boostered'].apply(lambda x: 1 if (x>bins[i] and x<=bins[i+1]) else 0)
                else:
                    df[bin_names[i]] = df['time vaccinated'].apply(lambda x: 1 if (x>bins[i] and x<=bins[i+1]) else 0)
            else:
                if(booster):
                    df[bin_names[i]] = df['time boostered'].apply(lambda x: 1 if (x>bins[i]) else 0)
                else:
                    df[bin_names[i]] = df['time vaccinated'].apply(lambda x: 1 if (x>bins[i]) else 0)
        if(booster==False):
            df[bin_names[len(bins)-1]] = df['time vaccinated'].apply(lambda x: 1 if (x>bins[len(bins)-1]) else 0)
    else:
        for i in range(len(bins)-1):
            df[bin_names[i]] = 0
        if(booster==False):
            df[bin_names[len(bins)-1]] = 0
    return df

In [ ]:
file_name = r"df_all.pkl"
df_all = pd.read_pickle(file_name)
df_all = df_all.drop_duplicates()
df_all = df_all.drop_duplicates(subset='seq_id')

file_name = r"df_vaccinated.pkl"
df_vaccinated = pd.read_pickle(file_name)
df_vaccinated = df_vaccinated.drop_duplicates()
df_vaccinated = df_vaccinated.drop_duplicates(subset='seq_id')

file_name = r"df_unvaccinated.pkl"
df_unvaccinated = pd.read_pickle(file_name)
df_unvaccinated = df_unvaccinated.drop_duplicates()
df_unvaccinated = df_unvaccinated.drop_duplicates(subset='seq_id')

file_name = r"df_boosters.pkl"
df_booster = pd.read_pickle(file_name)
df_booster = df_booster.drop_duplicates()
df_booster = df_booster.drop_duplicates(subset='seq_id')

df_vaccinated['time vaccinated'] = df_vaccinated['sample_date_lab'] - df_vaccinated['second_vaccine']
df_unvaccinated['time vaccinated'] = 0
df_booster['time vaccinated'] = df_booster['sample_date_lab'] - df_booster['second_vaccine']

df_vaccinated['time boostered'] = 0
df_unvaccinated['time boostered'] = 0
df_booster['time boostered'] = df_booster['sample_date_lab'] - df_booster['third_vaccine']

file_name = r"Hospitalization.pkl"
hospital = pd.read_pickle(file_name)

In [ ]:
# January 1st - 366
# February 1st - 397
# March 1st - 425
# April 1st - 456
# May 1st - 486
# June 1st - 517
# July 1st - 547
# August 1st - 578
# September 1st - 609
# October 1st - 639
# November 1st - 670
# December 1st - 700

earliest_sample_date = 544

latest_sample_date = 698

bins = [6, 30, 60, 120, 180]
booster_bins = [6, 30, 60, 120]


df_vaccinated = binning(df_vaccinated, bins, False, False, False)
df_booster = binning(df_booster, bins, True, False, False)
df_unvaccinated = binning(df_unvaccinated, bins, True, False, False)

df_vaccinated = binning(df_vaccinated, booster_bins, True, False, True)
df_booster = binning(df_booster, booster_bins, False, False, True)
df_unvaccinated = binning(df_unvaccinated, booster_bins, True, False, True)


df_vaccinated['vaccinated'] = 1
df_vaccinated['booster'] = 0

df_booster['vaccinated'] = 1
df_booster['booster'] = 1

df_unvaccinated['vaccinated'] = 0
df_unvaccinated['booster'] = 0

df_all_LR = pd.concat([df_vaccinated, df_unvaccinated, df_booster])
df_all_LR = df_all_LR.drop_duplicates(subset='seq_id')
df_all_LR = df_all_LR[df_all_LR['sample_date_lab']>=earliest_sample_date]
df_all_LR = df_all_LR[df_all_LR['sample_date_lab']<=latest_sample_date]
df_all_LR['relative_date_lab'] = df_all_LR['sample_date_lab'] - df_all_LR['sample_date_lab'].min()
df_all_LR['relative_date_lab_sqr'] = df_all_LR['relative_date_lab'] * df_all_LR['relative_date_lab']

df_all_LR['age 30-39'] = df_all_LR['age'].apply(lambda x: 1 if (x>=30 and x<40) else 0)
df_all_LR['age 40-49'] = df_all_LR['age'].apply(lambda x: 1 if (x>=40 and x<50) else 0)
df_all_LR['age 50-59'] = df_all_LR['age'].apply(lambda x: 1 if (x>=50 and x<60) else 0)
df_all_LR['age 60-69'] = df_all_LR['age'].apply(lambda x: 1 if (x>=60 and x<70) else 0)
df_all_LR['age 70-79'] = df_all_LR['age'].apply(lambda x: 1 if (x>=70 and x<80) else 0)
df_all_LR['age 80-89'] = df_all_LR['age'].apply(lambda x: 1 if (x>=80 and x<90) else 0)
df_all_LR['age 90-99'] = df_all_LR['age'].apply(lambda x: 1 if (x>=90 and x<100) else 0)
df_all_LR['age 100-109'] = df_all_LR['age'].apply(lambda x: 1 if (x>=100 and x<110) else 0)


df_all_LR = df_all_LR[df_all_LR['age']>=20]
df_all_LR = df_all_LR[df_all_LR['age']<=120]


print("immunosuppression 1 = ", len(df_all_LR[df_all_LR['immunosuppression']==1]))
print("immunosuppression 2 = ", len(df_all_LR[df_all_LR['immunosuppression']==2]))
df_all_LR = df_all_LR[df_all_LR['immunosuppression']==0]

print("hospitalized = ", len(df_all_LR[df_all_LR['seq_id'].isin(hospital['seq_id'])]))
print("not hospitalized = ", len(df_all_LR[~df_all_LR['seq_id'].isin(hospital['seq_id'])]))
print("hospitalized old = ", len(df_all_LR[df_all_LR['seq_id'].isin(hospital_old['seq_id'])]))
print("not hospitalized old = ", len(df_all_LR[~df_all_LR['seq_id'].isin(hospital_old['seq_id'])]))
print("all = ", len(df_all_LR))


df_vaccinated_change = binning(df_vaccinated, bins, False, True, False)
df_booster_change = binning(df_booster, bins, False, True, False)
df_unvaccinated_change = binning(df_unvaccinated, bins, True, True, False)

df_vaccinated_change = binning(df_vaccinated, booster_bins, False, True, True)
df_booster_change = binning(df_booster, booster_bins, False, True, True)
df_unvaccinated_change = binning(df_unvaccinated, booster_bins, True, True, True)


df_vaccinated_change['vaccinated'] = 1
df_vaccinated_change['booster'] = 0

df_booster_change['vaccinated'] = 1
df_booster_change['booster'] = 1

df_unvaccinated_change['vaccinated'] = 0
df_unvaccinated_change['booster'] = 0

df_all_LR_change = pd.concat([df_vaccinated_change, df_unvaccinated_change, df_booster_change])
df_all_LR_change = df_all_LR_change.drop_duplicates(subset='seq_id')
df_all_LR_change = df_all_LR_change[df_all_LR_change['sample_date_lab']>=earliest_sample_date]
df_all_LR_change = df_all_LR_change[df_all_LR_change['sample_date_lab']<=latest_sample_date]
df_all_LR_change['relative_date_lab'] = df_all_LR_change['sample_date_lab'] - df_all_LR_change['sample_date_lab'].min()
df_all_LR_change['relative_date_lab_sqr'] = df_all_LR_change['relative_date_lab'] * df_all_LR_change['relative_date_lab']

df_all_LR_change['age 30-39'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=30 and x<40) else 0)
df_all_LR_change['age 40-49'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=40 and x<50) else 0)
df_all_LR_change['age 50-59'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=50 and x<60) else 0)
df_all_LR_change['age 60-69'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=60 and x<70) else 0)
df_all_LR_change['age 70-79'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=70 and x<80) else 0)
df_all_LR_change['age 80-89'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=80 and x<90) else 0)
df_all_LR_change['age 90-99'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=90 and x<100) else 0)
df_all_LR_change['age 100-109'] = df_all_LR_change['age'].apply(lambda x: 1 if (x>=100 and x<110) else 0)


df_all_LR_change = df_all_LR_change[df_all_LR_change['age']>=20]
df_all_LR_change = df_all_LR_change[df_all_LR_change['age']<=120]

df_all_LR_change = df_all_LR_change[df_all_LR_change['immunosuppression']==0]

ylim = [6.7, -0.2]

reg_coef = binning_names(bins, False) + binning_names(booster_bins, True) + ['relative_date_lab', 'relative_date_lab_sqr', 'age 30-39', 'age 40-49', 'age 50-59', 'age 60-69', 'age 70-79', 'age 80-89', 'age 90-99', 'age 100-109', 'sex']
reg_coef_present = binning_names(bins, False) + binning_names(booster_bins, True)


for gene in ['N', 'RdRp', 'E']:
    
    if(gene=='RdRp'):
        name = "Figure 1"
        LR(reg_coef, reg_coef_present, df_all_LR, gene, ylim, name, (18, 18), df_all_LR_change)
    else:
        name = "Extended Data Figure 1"
        LR(reg_coef, reg_coef_present, df_all_LR, gene, ylim, name, (9, 9), df_all_LR_change)

print(len(df_all_LR))

In [ ]:
print("total # of samples:")
print(len(df_all_LR), len(df_all_LR[df_all_LR['sex']==1]), len(df_all_LR[df_all_LR['sex']==0]), df_all_LR['age'].mean(), df_all_LR['age'].std(), df_all_LR['RdRp'].mean(), df_all_LR['RdRp'].std())
print("")

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==0)]
print("unvax # of samples:")
print(len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std(), a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())
print("")
print("")
print("")


a = df_all_LR[((df_all_LR['7-30 after 2nd shot']==1) | (df_all_LR['31-60 after 2nd shot']==1) | (df_all_LR['61-120 after 2nd shot']==1) | (df_all_LR['121-180 after 2nd shot']==1) | (df_all_LR['>180 after 2nd shot']==1)) & ((df_all_LR['7-30 after booster']==0) & (df_all_LR['31-60 after booster']==0) & (df_all_LR['61-120 after booster']==0))]
print("2nd shot # of samples:")
print(len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print(a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())
print("")


a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==1) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==0)]
print('7-30: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('7-30: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==1) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==0)]
print('31-60: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('31-60: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==1) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==0)]
print('61-120: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('61-120: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==1) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==0)]
print('121-180: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('121-180: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==1) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==0)]
print('>180: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('>180: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())
print("")
print("")
print("")

a = df_all_LR[(df_all_LR['7-30 after booster']==1) | (df_all_LR['31-60 after booster']==1) | (df_all_LR['61-120 after booster']==1)]
print("booster # of samples:")
print(len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print(a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())
print("")

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==1) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==0)]
print('7-30: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('7-30: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==1) &(df_all_LR['61-120 after booster']==0)]
print('31-60: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('31-60: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())

a = df_all_LR[(df_all_LR['7-30 after 2nd shot']==0) &(df_all_LR['31-60 after 2nd shot']==0) &(df_all_LR['61-120 after 2nd shot']==0) &(df_all_LR['121-180 after 2nd shot']==0) &(df_all_LR['>180 after 2nd shot']==0) &(df_all_LR['7-30 after booster']==0) &(df_all_LR['31-60 after booster']==0) &(df_all_LR['61-120 after booster']==1)]
print('61-120: ', len(a), len(a[a['sex']==1]), len(a[a['sex']==0]), a['age'].mean(), a['age'].std())
print('61-120: ', a['RdRp'].mean(), a['RdRp'].std(), a['N'].mean(), a['N'].std(), a['E'].mean(), a['E'].std())

In [ ]:
plt.figure(figsize=[18/2.54, 18/2.54])
fig_2 = plt.hist([df_all_LR[df_all_LR['7-30 after booster']==1]['age'],
                  df_all_LR[df_all_LR['31-60 after booster']==1]['age'],
                  df_all_LR[df_all_LR['61-120 after booster']==1]['age']],
                 bins=[20,30,40,50,60,70,80,90,100], density=True)
plt.legend(labels=['7-30', '31-60', '61-120'], fontsize=13)
plt.ylabel('Fraction', size=17)
plt.xlabel('Age', size=17)
plt.yticks(ticks=[0,0.005,0.01,0.015,0.02,0.025], labels=[0,0.05,0.1,0.15,0.2,0.25], size=13)
plt.xticks(size=13)

name = 'Extended Data Figure 2'

plt.savefig(output_directory + r"png\\" + name + '.png', dpi=600, bbox_inches='tight')
plt.savefig(output_directory + r"eps\\" + name + '.eps', dpi=600, bbox_inches='tight')
plt.savefig(output_directory + r"jpg\\" + name + '.jpg', dpi=600, bbox_inches='tight')
plt.savefig(output_directory + r"tif\\" + name + '.tif', dpi=600, bbox_inches='tight')
plt.savefig(output_directory + r"svg\\" + name + '.svg', dpi=600, bbox_inches='tight')

export_array = pd.DataFrame(np.array([np.array(range(1,9)), np.array(fig_2[0][0]*10),
                                      np.array(fig_2[0][1]*10), np.array(fig_2[0][2]*10)]).T,
                           columns=['bar #', 'days 7-30', 'days 31-60', 'days 61-120'])
export_array.to_csv(output_directory + r"csv\\" + name + '.csv')

In [ ]:
time_const = 120

def fit_func(X, first_effect, second_effect, first_decay_const, second_decay_const, relative_date_coef, relative_date_lab_sqr_coef, age_30_39_coef, age_40_49_coef, age_50_59_coef, age_60_69_coef, age_70_79_coef, age_80_89_coef, age_90_99_coef, age_100_109_coef, sex_coef, const_coef):
    
    vaccinated, boosted, time_vaccinated, time_boosted, relative_date_lab, relative_date_lab_sqr, age_30_39, age_40_49, age_50_59, age_60_69, age_70_79, age_80_89, age_90_99, age_100_109, sex = X
    
    second_shot = vaccinated*np.exp(-first_decay_const*time_vaccinated/time_const) * first_effect
    third_shot = boosted * np.exp(-second_decay_const*time_boosted/time_const) * second_effect
    
    result = const_coef + second_shot + third_shot + relative_date_coef*relative_date_lab + relative_date_lab_sqr_coef*relative_date_lab_sqr + age_30_39_coef*age_30_39 + age_40_49_coef*age_40_49 + age_50_59_coef*age_50_59 + age_60_69_coef*age_60_69 + age_70_79_coef*age_70_79 + age_80_89_coef*age_80_89 + age_90_99_coef*age_90_99 + age_100_109_coef*age_100_109 + sex_coef*sex
    return result


df_fit = df_all_LR
df_fit = df_fit[['vaccinated', 'booster', 'time vaccinated', 'time boostered', 'relative_date_lab',
             'relative_date_lab_sqr', 'age 30-39', 'age 40-49','age 50-59', 'age 60-69', 'age 70-79',
             'age 80-89', 'age 90-99', 'age 100-109', 'sex', 'RdRp', 'N', 'E']].dropna()
variables = ['vaccinated', 'booster', 'time vaccinated', 'time boostered', 'relative_date_lab',
             'relative_date_lab_sqr', 'age 30-39', 'age 40-49','age 50-59', 'age 60-69', 'age 70-79',
             'age 80-89', 'age 90-99', 'age 100-109', 'sex']

first_effect_array = np.array([])
second_effect_array = np.array([])
first_decay_const_array = np.array([])
second_decay_const_array = np.array([])

for gene in ['RdRp', 'N', 'E']:
    print('')
    print('')
    print(gene)
    for i in range(5000):

        df_fit_bootstrap = df_fit.sample(replace=True, frac=1)

        popt, pcov = curve_fit(fit_func, df_fit_bootstrap[variables].values.T, df_fit_bootstrap[gene])

        first_effect_array = np.append(first_effect_array, popt[0])
        second_effect_array = np.append(second_effect_array, popt[1])
        first_decay_const_array = np.append(first_decay_const_array, popt[2])
        second_decay_const_array = np.append(second_decay_const_array, popt[3])
        
    print('first_effect_0.025: ', np.percentile(first_effect_array, 2.5))
    print('first_effect_0.975: ', np.percentile(first_effect_array, 97.5))
    print('')
    
    print('second_effect_0.025: ', np.percentile(second_effect_array, 2.5))
    print('second_effect_0.975: ', np.percentile(second_effect_array, 97.5))
    print('')
          
    print('first_decay_const_0.025: ', time_const/np.percentile(first_decay_const_array, 2.5))
    print('first_decay_const_0.975: ', time_const/np.percentile(first_decay_const_array, 97.5))
    print('')
    
    print('second_decay_const_0.025: ', time_const/np.percentile(second_decay_const_array, 2.5))
    print('second_decay_const_0.975: ', time_const/np.percentile(second_decay_const_array, 97.5))
    print('')

In [ ]:
df_all_LR = df_all_LR[~df_all_LR['seq_id'].isin(hospital['seq_id'])]
df_all_LR = df_all_LR[~df_all_LR['seq_id'].isin(hospital_old['seq_id'])]

df_all_LR_change = df_all_LR_change[~df_all_LR_change['seq_id'].isin(hospital['seq_id'])]
df_all_LR_change = df_all_LR_change[~df_all_LR_change['seq_id'].isin(hospital_old['seq_id'])]

ylim = [6.7, -0.2]

for gene in ['N', 'RdRp', 'E']:
    
    if(gene=='RdRp'):
        name = "Extended Data Figure 3"
        LR(reg_coef, reg_coef_present, df_all_LR, gene, ylim, name, (18, 18), df_all_LR_change)
    else:
        name = "Extended Data Figure 3"
        LR(reg_coef, reg_coef_present, df_all_LR, gene, ylim, name, (9, 9), df_all_LR_change)